In [35]:
import json
import requests
import pandas as pd
from pathlib import Path
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path
from sklearn import cluster, covariance, manifold
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import warnings
warnings.filterwarnings('ignore')


In [36]:
events=pd.read_csv(
    Path('China_events_2021.csv'),
    index_col='DATE',
    parse_dates=True,
    infer_datetime_format=True
)
events=events.fillna(0)
events.head()

,NEWS,CATEGORY,INDICATOR
DATE,,,
2021-05-18,China Bans Institutions From Crypto Business,China,-1
2021-05-21,China Crackdown on Mining & Trading,China,-1
2021-06-21,China Intensifies Mining Crackdown,China,-1
2021-09-20,Evergrande Crisis Hits,China,-1
2021-09-27,China: All Crypto Business is Illegal,China,-1


In [37]:
events=events.drop(['NEWS', 'CATEGORY'], axis=1)
events.head()

,INDICATOR
DATE,
2021-05-18,-1
2021-05-21,-1
2021-06-21,-1
2021-09-20,-1
2021-09-27,-1


In [38]:
prices=pd.read_csv(
    Path('crypto_market_caps.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True

)
prices.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
date,,,,,,,,,,,,,,,,,,,,,
2021-05-29,0.000000,-0.087474,-0.084921,-0.178955,-0.069173,-0.075526,-0.116392,-0.141745,-0.110146,-0.081747,...,-0.086776,-0.107659,-0.161449,-0.144139,-0.127478,-0.204751,-0.148758,-0.141004,-0.074100,-0.154125
2021-05-30,0.231838,-0.029180,-0.077416,-0.050995,-0.020782,-0.021363,-0.048031,-0.045486,-0.063362,-0.056490,...,-0.101236,-0.070963,-0.058184,-0.062235,-0.049803,-0.049462,-0.048317,-0.080900,-0.092101,-0.066768
2021-05-31,0.238859,0.067797,0.092618,-0.037861,0.022546,0.022048,0.035723,0.030508,0.015360,0.099605,...,0.095878,0.008402,0.111175,0.081288,0.070708,0.119675,0.043991,0.033885,0.081935,0.058975
2021-06-01,-0.059507,0.106957,0.026690,-0.004131,-0.004840,0.045575,0.130062,0.145793,0.143006,0.109167,...,0.102625,0.202528,0.106781,0.178813,0.082806,0.195291,0.104627,0.063740,0.081550,0.123031
2021-06-02,-0.205798,-0.007821,-0.032677,0.333030,-0.024114,-0.014893,-0.025359,-0.050999,-0.015254,0.000810,...,-0.030430,-0.194083,-0.023898,-0.023796,0.002862,-0.057441,-0.031295,0.050371,0.002730,-0.042151


In [39]:
combined=pd.merge(prices,events,left_index=True, right_index=True)
combined.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,INDICATOR
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792,-1
2021-09-20,0.092967,-0.021709,0.020197,0.010053,-0.034154,-0.018059,-0.026761,-0.098414,-0.034175,-0.031707,...,-0.101968,-0.048078,-0.039046,-0.022769,-0.001677,-0.078545,0.126641,0.000000,-0.088141,-1
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621,-1
2021-05-18,0.000000,-0.004831,-0.032908,0.126400,0.000000,-0.060157,-0.079908,0.007466,-0.077167,-0.105774,...,-0.047775,-0.059087,0.127019,-0.053584,-0.786075,-0.044020,0.134674,-0.058431,-0.032635,-1
2021-05-21,0.000000,0.077480,-0.048847,0.102132,2.364960,0.088701,0.091215,0.226731,0.091230,0.168138,...,0.097783,0.069360,0.080627,0.087000,0.261868,0.152568,0.247294,0.098162,1.020536,-1


In [40]:
combined = combined.dropna(axis=0, how='all')
combined = combined.dropna(axis=1)
combined

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,INDICATOR
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792,-1
2021-09-20,0.092967,-0.021709,0.020197,0.010053,-0.034154,-0.018059,-0.026761,-0.098414,-0.034175,-0.031707,...,-0.101968,-0.048078,-0.039046,-0.022769,-0.001677,-0.078545,0.126641,0.000000,-0.088141,-1
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621,-1
2021-05-18,0.000000,-0.004831,-0.032908,0.126400,0.000000,-0.060157,-0.079908,0.007466,-0.077167,-0.105774,...,-0.047775,-0.059087,0.127019,-0.053584,-0.786075,-0.044020,0.134674,-0.058431,-0.032635,-1
2021-05-21,0.000000,0.077480,-0.048847,0.102132,2.364960,0.088701,0.091215,0.226731,0.091230,0.168138,...,0.097783,0.069360,0.080627,0.087000,0.261868,0.152568,0.247294,0.098162,1.020536,-1


In [41]:
combined_new=combined.drop(['INDICATOR'], axis=1)
combined_new

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,0.047777,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792
2021-09-20,0.092967,-0.021709,0.020197,0.010053,-0.034154,-0.018059,-0.026761,-0.098414,-0.034175,-0.031707,...,-0.021130,-0.101968,-0.048078,-0.039046,-0.022769,-0.001677,-0.078545,0.126641,0.000000,-0.088141
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.212313,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621
2021-05-18,0.000000,-0.004831,-0.032908,0.126400,0.000000,-0.060157,-0.079908,0.007466,-0.077167,-0.105774,...,-0.075334,-0.047775,-0.059087,0.127019,-0.053584,-0.786075,-0.044020,0.134674,-0.058431,-0.032635
2021-05-21,0.000000,0.077480,-0.048847,0.102132,2.364960,0.088701,0.091215,0.226731,0.091230,0.168138,...,0.080968,0.097783,0.069360,0.080627,0.087000,0.261868,0.152568,0.247294,0.098162,1.020536


In [42]:
variation = combined_new
variation.shape

(5, 52)

In [43]:
column_names= list(combined_new)

In [44]:
# calculate mean and variance for each token
annual_returns = []
variance = []
for x in column_names:
    annual_returns.append(combined_new[x].mean()*252)
    variance.append(combined_new[x].std()*np.sqrt(252))
    
    

In [45]:
stats=pd.DataFrame([annual_returns,variance])

In [46]:
stats_df=stats.T

In [47]:
stats_df.columns=('annual_return','variance')

In [48]:
stats_df.index=column_names

In [49]:
stats_df.head()

,annual_return,variance
spell-token,4.218803,0.787767
illuvium,3.751228,0.613360
polyswarm,-3.251798,0.869035
hackenai,12.175299,1.048819
mercurial,115.387471,16.924705


In [50]:
# Create a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [51]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(stats_df)
    inertia.append(k_model.inertia_)

In [52]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,17146.046555
1,2,5019.873955
2,3,2505.326803
3,4,1562.059389
4,5,802.423620


In [53]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [54]:
# Create and initialize the K-means model instance for 4 clusters
model = KMeans(n_clusters=4, random_state=1)

In [55]:
# Fit the data to the instance of the model
model.fit(stats_df)

# Make predictions about the data clusters using the trained model
crypto_clusters = model.predict(stats_df)
crypto_clusters

array([0, 0, 0, 3, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 3, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 3, 2, 0, 3, 3,
       3, 3, 3, 0, 0, 2, 0, 2], dtype=int32)

In [56]:
len(crypto_clusters)

52

In [57]:
crypto_clusters_df = pd.DataFrame(crypto_clusters)
crypto_clusters_df.head()

,0
0,0
1,0
2,0
3,3
4,1


In [58]:
stats_df['crypto_cluster']=crypto_clusters
stats_df['names']=column_names
stats_df


,annual_return,variance,crypto_cluster,names
spell-token,4.218803,0.787767,0,spell-token
illuvium,3.751228,0.613360,0,illuvium
polyswarm,-3.251798,0.869035,0,polyswarm
hackenai,12.175299,1.048819,3,hackenai
mercurial,115.387471,16.924705,1,mercurial
bitcoin,1.874433,0.869962,0,bitcoin
ethereum,2.668683,1.041299,0,ethereum
solana,6.521749,1.908718,3,solana
polkadot,-2.334449,1.040205,0,polkadot
cardano,1.821264,1.621605,0,cardano


In [59]:
# Plot the crypto clusters Data Frame
plot=stats_df.hvplot.scatter(x="annual_return", y="variance", by="crypto_cluster", hover=True, hover_cols=['index'])
#labels=stats_df.hvplot.labels(x='annual_return', y='variance',by='crypto_cluster', text='names',text_baseline='bottom', hover=True)
#plot*labels
plot

:NdOverlay   [crypto_cluster]
   :Scatter   [annual_return]   (variance,index)

In [60]:
correlations_df=combined_new.corr()
btc_correlations=correlations_df.loc['bitcoin']

In [61]:
btc_correlations=btc_correlations.sort_values()

btc_correlations.hvplot.scatter()

:Scatter   [index]   (bitcoin)

In [62]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(correlations_df, annot=True, linewidths=.5)


In [63]:
correlations_df

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
spell-token,1.000000,-0.555308,0.714198,-0.040506,-0.195356,-0.171863,-0.270772,-0.479309,-0.341251,-0.273920,...,-0.062949,0.039597,-0.638064,-0.801031,-0.397422,0.119615,-0.472648,0.174264,-0.116900,-0.245829
illuvium,-0.555308,1.000000,-0.566382,0.257398,0.906823,0.841038,0.788364,0.953525,0.919302,0.914345,...,0.264115,0.083530,0.560056,0.393177,0.792674,0.496092,0.976881,0.483042,0.851159,0.928868
polyswarm,0.714198,-0.566382,1.000000,-0.068722,-0.371214,-0.103369,-0.102732,-0.452580,-0.501373,-0.384587,...,0.494397,0.665470,-0.272979,-0.451094,-0.160194,0.131018,-0.383233,-0.315211,-0.241910,-0.373265
hackenai,-0.040506,0.257398,-0.068722,1.000000,0.463113,0.005390,-0.192807,0.525363,-0.043979,-0.009276,...,-0.239554,0.096661,-0.508676,0.568178,-0.221759,-0.451237,0.299060,0.690658,-0.014055,0.449780
mercurial,-0.195356,0.906823,-0.371214,0.463113,1.000000,0.824961,0.682672,0.936979,0.854183,0.877077,...,0.159906,0.068502,0.219529,0.199175,0.627380,0.481009,0.913124,0.762508,0.869591,0.997403
bitcoin,-0.171863,0.841038,-0.103369,0.005390,0.824961,1.000000,0.969220,0.772401,0.896597,0.947914,...,0.625237,0.396060,0.612523,-0.026310,0.930925,0.853451,0.900767,0.314020,0.965558,0.846067
ethereum,-0.270772,0.788364,-0.102732,-0.192807,0.682672,0.969220,1.000000,0.669441,0.864149,0.907172,...,0.718126,0.435532,0.778928,-0.028350,0.988432,0.890118,0.843499,0.083298,0.910660,0.715629
solana,-0.479309,0.953525,-0.452580,0.525363,0.936979,0.772401,0.669441,1.000000,0.789569,0.806405,...,0.234456,0.182984,0.360084,0.508327,0.663094,0.335890,0.962916,0.604807,0.757394,0.955158
polkadot,-0.341251,0.919302,-0.501373,-0.043979,0.854183,0.896597,0.864149,0.789569,1.000000,0.990120,...,0.272261,-0.030160,0.596265,0.018948,0.838830,0.733097,0.881838,0.435441,0.958137,0.864223
cardano,-0.273920,0.914345,-0.384587,-0.009276,0.877077,0.947914,0.907172,0.806405,0.990120,1.000000,...,0.371885,0.091775,0.587220,-0.004911,0.872787,0.782533,0.906514,0.436558,0.985740,0.888629


In [64]:
# Plot the DataFrame
# Visualization
plt.figure(1, facecolor='w', figsize=(12, 9))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')


#Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)


# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.nipy_spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)

# a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)


# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):
    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.nipy_spectral(label / float(n_labels)),
                       alpha=.6))
plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())
plt.show()

NameError: name 'edge_model' is not defined